In [1]:
# 导入相关包
# 测试环境 K80 pytorch1.10
import torch
import torchvision 
from tqdm import tqdm
import matplotlib.pyplot as plt

/root/miniconda3/envs/myconda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


详细使用教程可以查看[矩池云入门手册](https://matpool.com/supports/doc-quick-start/)

## 一、数据预处理
### 1.1 构建全局变量

In [2]:
#如果网络能在GPU中训练，就使用GPU；否则使用CPU进行训练
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(device)

cuda:0


In [3]:
#这个函数包括了两个操作：将图片转换为张量，以及将图片进行归一化处理
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),
                                torchvision.transforms.Normalize(mean = [0.5],std = [0.5])])

In [4]:
# 设置了每个包中的图片数据个数
BATCH_SIZE = 64
EPOCHS = 10

### 1.2 加载构建训练和测试数据集

In [5]:
# 加载训练和测试数据
train_dataset = torchvision.datasets.MNIST('/mnt/MyMNIST/',train = True,transform = transform)
test_dataset = torchvision.datasets.MNIST('/mnt/MyMNIST/',train = False,transform = transform)

In [6]:
# 建立一个数据迭代器
# 装载训练集
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=BATCH_SIZE,
                                           shuffle=True)
# 装载测试集
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=BATCH_SIZE,
                                          shuffle=True)

## 二、数据训练部分
### 2.1 构建模型

In [7]:
# 一个简单的卷积神经网络
class Net(torch.nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.model = torch.nn.Sequential(
            #The size of the picture is 28x28
            torch.nn.Conv2d(in_channels = 1,out_channels = 16,kernel_size = 3,stride = 1,padding = 1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size = 2,stride = 2),
            
            #The size of the picture is 14x14
            torch.nn.Conv2d(in_channels = 16,out_channels = 32,kernel_size = 3,stride = 1,padding = 1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size = 2,stride = 2),
            
            #The size of the picture is 7x7
            torch.nn.Conv2d(in_channels = 32,out_channels = 64,kernel_size = 3,stride = 1,padding = 1),
            torch.nn.ReLU(),
            
            torch.nn.Flatten(),
            torch.nn.Linear(in_features = 7 * 7 * 64,out_features = 128),
            torch.nn.ReLU(),
            torch.nn.Linear(in_features = 128,out_features = 10),
            torch.nn.Softmax(dim=1)
        )
        
    def forward(self,input):
        output = self.model(input)
        return output

In [8]:
# 如果运行出现 'tqdm' object has no attribute 'sp'
# 就pip安装下ipywidgets pip install ipywidgets

In [9]:
# 构建模型实例
net = Net()
# to() 方法用于将张量放入到指定的设备（如CPU或GPU中），不同设备的张量是无法进行运算的
print(net.to(device))

Net(
  (model): Sequential(
    (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): Flatten(start_dim=1, end_dim=-1)
    (9): Linear(in_features=3136, out_features=128, bias=True)
    (10): ReLU()
    (11): Linear(in_features=128, out_features=10, bias=True)
    (12): Softmax(dim=1)
  )
)


### 2.2 构建迭代器与损失函数

In [10]:
# 交叉熵损失来作为损失函数
# Adam迭代器
loss_fun = torch.nn.CrossEntropyLoss() 
optimizer = torch.optim.Adam(net.parameters())

### 2.3 构建训练循环

In [ ]:
history = {'Test Loss':[],'Test Accuracy':[]}
for epoch in range(1,EPOCHS + 1):
    process_bar = tqdm(train_loader,unit = 'step')
    net.train(True)
    for step,(train_imgs,labels) in enumerate(process_bar):
        train_imgs = train_imgs.to(device)
        labels = labels.to(device)

        net.zero_grad()
        outputs = net(train_imgs)
        loss = loss_fun(outputs,labels)
        predictions = torch.argmax(outputs, dim = 1)
        accuracy = torch.true_divide(torch.sum(predictions == labels), labels.shape[0])
        loss.backward()

        optimizer.step()
        process_bar.set_description("[%d/%d] Loss: %.4f, Acc: %.4f" % 
                                   (epoch,EPOCHS,loss.item(),accuracy.item()))
        
        if step == len(process_bar)-1:
            correct,total_loss = 0,0
            net.train(False)
            with torch.no_grad():
                for test_imgs,labels in test_loader:
                    test_imgs = test_imgs.to(device)
                    labels = labels.to(device)
                    outputs = net(test_imgs)
                    loss = loss_fun(outputs,labels)
                    predictions = torch.argmax(outputs,dim = 1)

                    total_loss += loss
                    correct += torch.sum(predictions == labels)

                test_accuracy = torch.true_divide(correct, (BATCH_SIZE * len(test_loader)))
                test_loss = torch.true_divide(total_loss, len(test_loader))
                history['Test Loss'].append(test_loss.item())
                history['Test Accuracy'].append(test_accuracy.item())
            
            process_bar.set_description("[%d/%d] Loss: %.4f, Acc: %.4f, Test Loss: %.4f, Test Acc: %.4f" % 
                                   (epoch,EPOCHS,loss.item(),accuracy.item(),test_loss.item(),test_accuracy.item()))
    process_bar.close()


[1/10] Loss: 1.5112, Acc: 1.0000, Test Loss: 1.4895, Test Acc: 0.9677: 100%|██████████| 938/938 [00:25<00:00, 36.43step/s]
[2/10] Loss: 1.4612, Acc: 1.0000, Test Loss: 1.4831, Test Acc: 0.9735: 100%|██████████| 938/938 [00:25<00:00, 36.34step/s]
[3/10] Loss: 1.4624, Acc: 1.0000, Test Loss: 1.4802, Test Acc: 0.9759: 100%|██████████| 938/938 [00:26<00:00, 36.05step/s]
[4/10] Loss: 1.4612, Acc: 1.0000, Test Loss: 1.4754, Test Acc: 0.9813: 100%|██████████| 938/938 [00:26<00:00, 36.07step/s]
[5/10] Loss: 1.4612, Acc: 1.0000, Test Loss: 1.4727, Test Acc: 0.9841: 100%|██████████| 938/938 [00:26<00:00, 35.91step/s]
[6/10] Loss: 1.4612, Acc: 1.0000, Test Loss: 1.4724, Test Acc: 0.9842: 100%|██████████| 938/938 [00:25<00:00, 36.16step/s]
[7/10] Loss: 1.4612, Acc: 1.0000, Test Loss: 1.4724, Test Acc: 0.9839: 100%|██████████| 938/938 [00:25<00:00, 36.75step/s]
[8/10] Loss: 1.4614, Acc: 1.0000:  12%|█▏        | 115/938 [00:02<00:20, 40.03step/s]

## 三、数据预后处理部分

### 3.1 训练结果可视化

In [ ]:
#对测试Loss进行可视化
plt.plot(history['Test Loss'],label = 'Test Loss')
plt.legend(loc='best')
plt.grid(True)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

#对测试准确率进行可视化
plt.plot(history['Test Accuracy'],color = 'red',label = 'Test Accuracy')
plt.legend(loc='best')
plt.grid(True)
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.show()

##### 参考
> [PyTorch手写数字识别(MNIST数据集)](https://blog.csdn.net/weixin_44613063/article/details/90815082)<br>
> [Pytorch入门——手把手教你MNIST手写数字识别](https://blog.csdn.net/NikkiElwin/article/details/112980305)
